# --------------------------使用pytorch搭建rexnet18模型----------------------

- 先构建残差块
- 然后逐层搭建模型
- 实例化模型

In [20]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models

In [21]:
"""定义残差连接"""
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_conv=False, strides=1):
        super().__init__()
        self.fn1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.fn2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1, stride=1)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        
        # 定义是否需要使用1x1卷积来改变通道维度
        if use_conv:
            self.fn3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.fn3 = None
        
    # 定义模型前向传播函数
    def forward(self, x):
        output = F.relu(self.bn1(self.fn1(x))) # 卷积+批量归一化+激活函数
        output = self.bn2(self.fn2(output)) # 卷积+批量归一化
        if self.fn3:
            x = self.fn3(x)
        output += x
        return F.relu(output)

In [22]:
"""定义残差块"""
def resnetblock(input_channels, num_channels, num_residual, first=False):
    blk = []
    for i in range(num_residual):
        if i == 0 and not first:
            blk.append(Residual(input_channels, num_channels, use_conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels, use_conv=False))
    
    return blk

In [23]:
"""一层一层搭建模型"""
def resnet18(output):
    b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3), nn.BatchNorm2d(64),nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    b2 = nn.Sequential(*resnetblock(64, 64, 2, first=True)) # "解包,将序列所有元素作为单独的参数,并创建一个新的神经网络层序列。
    b3 = nn.Sequential(*resnetblock(64, 128, 2))
    b4 = nn.Sequential(*resnetblock(128, 256, 2))
    b5 = nn.Sequential(*resnetblock(256, 512, 2))

# 将所有层汇总，变成一个整体模型
    model = nn.Sequential(b1, b2, b3, b4, b5, 
                      nn.AdaptiveAvgPool2d((1, 1)),# 全局平均池化层
                      nn.Flatten(), # 将张量转换成一维
                      nn.Linear(512, output) # 全连接层对应输出分类
                     )
    return model
    

"""定义使用GPU进行计算"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

# 查看模型可学习的参数量
total_params = 0
# 遍历模型的所有参数，并累加它们的元素数量
for param in model.parameters():
    # 参数元素数量等于其形状中的元素个数之积
    num_params = param.numel()
    total_params += num_params

print(f"Total parameters: {total_params}")